<a href="https://colab.research.google.com/github/mamoan/NOVA_DL_home_exercise/blob/main/6_Post_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6 - Post-processing
## Clean-up (removing duplicates and edge detections) 📝-->🧹📝
## Converting YOLO predictions from image to map space  🧹📝--> 🗺️


In [ ]:
path_to_predicted_list = ["/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_galbyveien_20230504_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_braatan_40m_20230605_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_krakstad_202304_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_ortho_hobol_042222_mavic_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/test/images/predict_tiles_ML", "/content/drive/MyDrive/NOVA_course_home_exercise/data/annotated_data/test/images/predict_tiles_ML_small"]
path_to_predicted_list1 = ["/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_galbyveien_20230504_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_braatan_40m_20230605_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_krakstad_202304_sun", "/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_ortho_hobol_042222_mavic_sun"]
buffer_size_m=1

In [ ]:
path_to_tile_index_list = [path_to_predicted_list1[0]+"/galbyveien_20230504_sun_tile_index.shp", path_to_predicted_list1[1]+"/braatan_40m_20230605_sun_tile_index.shp", path_to_predicted_list1[2]+"/krakstad_202304_sun_tile_index.shp", path_to_predicted_list1[3]+"/ortho_hobol_042222_mavic_sun_tile_index.shp"]

In [ ]:
path_to_tile_index_list

## 6.1 load required libraries

In [ ]:
!pip install geopandas
# general libraries
import os, glob
from pathlib import Path

# Geospatial libraries
import geopandas as gpd
from osgeo import gdal, osr
from shapely.geometry import Polygon


In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 6.2 Get necessary files


In [ ]:
gtiffs = [None] * 4
labels = [None] * 4
labels_small = [None] * 4

In [ ]:
path_to_predicted_list1

['/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_galbyveien_20230504_sun',
 '/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_braatan_40m_20230605_sun',
 '/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_krakstad_202304_sun',
 '/content/drive/MyDrive/NOVA_course_home_exercise/data/tiles/10m_ortho_hobol_042222_mavic_sun']

In [ ]:
path_to_orthomosaics = ["/content/drive/MyDrive/NOVA_course_home_exercise/data/orthomosaics/test_data/galbyveien_20230504_sun.tif", "/content/drive/MyDrive/NOVA_course_home_exercise/data/orthomosaics/test_data/braatan_40m_20230605_sun.tif", "/content/drive/MyDrive/NOVA_course_home_exercise/data/orthomosaics/test_data/10m_krakstad_202304_sun.tif", "/content/drive/MyDrive/NOVA_course_home_exercise/data/orthomosaics/test_data/10m_ortho_hobol_042222_mavic_sun.tif"]

In [ ]:
for i in range(4):
  gtiffs[i] = glob.glob(path_to_predicted_list1[i] + "/*.tif") # all tif files
  # gtiffs[i] = path_to_orthomosaics[i]
  labels[i] = glob.glob(path_to_predicted_list1[i] + "/predict_tiles/labels/*.txt") # all label files
  labels_small[i] = glob.glob(path_to_predicted_list1[i] + "/predict_tiles_small/labels/*.txt") # all label files
  print("There a total of "+str(len(gtiffs[i]))+" tif files")
  print("There a total of "+str(len(labels[i]))+" label files for the full model")
  print("There a total of "+str(len(labels_small[i]))+" label files for the small model")

There a total of 580 tif files
There a total of 578 label files for the full model
There a total of 579 label files for the small model
There a total of 416 tif files
There a total of 369 label files for the full model
There a total of 363 label files for the small model
There a total of 217 tif files
There a total of 208 label files for the full model
There a total of 212 label files for the small model
There a total of 189 tif files
There a total of 188 label files for the full model
There a total of 187 label files for the small model


In [ ]:
# load tile index
tile_index_list = [None] * 4
tile_index_list = [gpd.read_file(path_to_tile_index_list[i]) for i in range(4)]

In [ ]:
src_ds = [None] * 4
src_ds = [gdal.Open(gtiffs[i][0]) for i in range(4)] # get raster datasource. Only the first tile is read but that is okay, because we are interested in the dimensions of the tiles and they should be the same.

In [ ]:
tile_size_m = [None] * 4
tile_size_px = [None] * 4
proj = [None] * 4
EPSG_code = [None] * 4

In [ ]:
for i in range(4):
  _, xres, _, _, _, yres  = src_ds[i].GetGeoTransform() # get pixel size in meters
  tile_size_m[i]=round(src_ds[i].RasterXSize*xres)
  tile_size_px[i]= round(tile_size_m[i]/abs(xres)) # calculate the tile size in pixels. The tile size is different for the different drone mosaics.
  proj[i] = osr.SpatialReference(wkt=src_ds[i].GetProjection())
  EPSG_code[i]= proj[i].GetAttrValue('AUTHORITY',1)

  print("Resolution: "+str(round(xres,2))+" m")
  print("EPSG: "+str(EPSG_code[i]))

Resolution: 0.01 m
EPSG: 25832
Resolution: 0.01 m
EPSG: 25832
Resolution: 0.01 m
EPSG: 25832
Resolution: 0.01 m
EPSG: 25832


## 6.2 Parse from YOLO (image space) to UTM coordinates (map space)

To do this we have to make a nested loop that iterates through each predicted label .txt file and through each row (i.e. predicted bounding box) in the .txt file.

But first we have to write a function to parse yolo coordinates (x,y,w,h) to x1, y1, x2, y2

In [ ]:
##########################################################################################################################################################################
# YOLO to x1, y1, x2, y2 parsers: converts (x, y, width, height) YOLO format to (x1, y1, x2, y2)  format.
# arguments:
# - label_file: file with YOLO predictions(s) inside including: class, x, y, width, height, probabilities
# - img_width - width of input image in pixels
# - img_height - height of input image in pixels
#Returns:
# - a file with a row per predicted bounding box and the following columns: class, x1, y1, x2, y2, probability (note that the coordinates are still in image coordinates and NOT GEOGRAPHICAL ONES)

##########################################################################################################################################################################

def yolo2xy(label_file, img_width, img_height):
    """
    Definition:
Parameters:
"""
    lfile = open(label_file)
    coords = []
    all_coords = []
    for line in lfile:
        l = line.split(" ")
        label=list(map(float, list(map(float, l[0]))))
        probabs=(l[5])
        #print(probabs)
        coords = list(map(float, list(map(float, l[1:6]))))
        x1 = float(img_width) * (2.0 * float(coords[0]) - float(coords[2])) / 2.0
        y1 = float(img_height) * (2.0 * float(coords[1]) - float(coords[3])) / 2.0
        x2 = float(img_width) * (2.0 * float(coords[0]) + float(coords[2])) / 2.0
        y2 = float(img_height) * (2.0 * float(coords[1]) + float(coords[3])) / 2.0
        tmp = [int(label[0]), int(x1), int(y1), int(x2), int(y2), float(coords[4])]
        all_coords.append(list(tmp))
    lfile.close()
    return all_coords


In [ ]:
labels[2]

Now we are ready to run the code bellow, which includes the above function. Labels are the labels for the full model, so I must remember to also run with labels from the small model later.

In [ ]:
all_bboxes = None
iter_all=0 # setup counter
j = 0 # This is changed according to which area I want to do the parsing for
labels = labels_small # Changed according to if I want to use prediction from the full or small model
for lab in range(len(labels[j])):
    print(str(round(lab/len(labels[j])*100))+" % done!", end="\r")
    # Define one label file and select the corresponding geotiff image
    label_file=labels[j][lab]
    label_file_name=Path(label_file).stem # ortho name
    for p in gtiffs[j]:
        if Path(p).stem ==label_file_name:
            gtiff_file=p

    # determing image width and height
    # try:
    r = gdal.Open(gtiff_file)
    # except:
    #   print("no file")
    #   continue

    img_width=r.RasterXSize
    img_height=r.RasterYSize

    # Convert from yolo coordinates to x1, y1, x2, y2,
    coords = yolo2xy(label_file, img_width, img_height) # class, x1, y1, x2, y2, probability

    # Convert from image to geographical coordinates
    ## select tile polygon (from tile index shapefile) that corresponds to the label_file_name
    # the other files are required by the gpd readfile


    # tile_index is <class 'geopandas.geodataframe.GeoDataFrame'>
    one_tile=tile_index_list[j][tile_index_list[j]['ID']==label_file_name+".tif"] # Select tile in tile_index that has ID equal to label_file_name

    ## get tile bounding box geographical coordinates (UTM)
    one_tile_XminUTM=one_tile.total_bounds[0]
    one_tile_YminUTM=one_tile.total_bounds[1]
    one_tile_XmaxUTM=one_tile.total_bounds[2]
    one_tile_YmaxUTM=one_tile.total_bounds[3]

    ## take inner buffer equal to the buffer_size_m
    one_tile_innerB= one_tile
    one_tile_innerB['geometry'] = one_tile_innerB.geometry.buffer(-(buffer_size_m/2))

    ## get inner tile bounding boxes
    one_tile_inner_XminUTM=one_tile_innerB.total_bounds[0]
    one_tile_inner_YminUTM=one_tile_innerB.total_bounds[1]
    one_tile_inner_XmaxUTM=one_tile_innerB.total_bounds[2]
    one_tile_inner_YmaxUTM=one_tile_innerB.total_bounds[3]

    # Now iterate through each bounding box and assign UTM coordinates and create a shapefile
    bboxes_tile = None
    for i in coords:
        # print("inside coords")
        # Convert bounding box coordinates from image to geographical coords
        X1_UTM=(i[1]*xres)+one_tile_XminUTM
        Y1_UTM=(i[2]*yres)+one_tile_YminUTM+tile_size_m[j]
        X2_UTM=(i[3]*xres)+one_tile_XminUTM
        Y2_UTM=(i[4]*yres)+one_tile_YminUTM+tile_size_m[j]

        # skip bounding box if its centroid is NOT within the inner tile (removing the overlap)
        X_UTM= (X1_UTM+X2_UTM)/2
        Y_UTM= (Y1_UTM+Y2_UTM)/2
        if X_UTM<one_tile_inner_XminUTM or X_UTM>one_tile_inner_XmaxUTM or Y_UTM<one_tile_inner_YminUTM or Y_UTM>one_tile_inner_YmaxUTM:
            #print("continue break")
            continue


        # Create polygon shape from geographical coords
        lat_point_list = [Y1_UTM, Y1_UTM, Y2_UTM, Y2_UTM, Y1_UTM]
        lon_point_list = [X1_UTM, X2_UTM, X2_UTM, X1_UTM, X1_UTM]
        # try:
        polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
        # except:
        #   print("not sure what is going on")
        #   continue

        crs = {'init': 'epsg:'+EPSG_code[j]}
        data= {'class': [i[0]], 'prob': [i[5]]}
        bbox = gpd.GeoDataFrame(data, crs=crs, geometry=[polygon_geom])

        if (bboxes_tile is None):
            bboxes_tile = bbox
        else:
            bboxes_tile = bboxes_tile.append(bbox)

    # cleanup boxes (removing overlapping ones)
    if bboxes_tile is not None:
        # clean_boxes = bboxes_tile #cleanUp_boudingBoxes(bboxes_tile, iou_thresh)
        if (all_bboxes is None):
            all_bboxes = bboxes_tile
        else :
            all_bboxes = all_bboxes.append(bboxes_tile)


In [ ]:
ortho_name= os.path.basename(path_to_predicted_list1[j]) # Check that the code was run for the intended area
print(ortho_name)

10m_galbyveien_20230504_sun


## 6.3 Export predictions shapefile

In [ ]:
ortho_name= os.path.basename(path_to_predicted_list1[j])
all_bboxes.to_file(path_to_predicted_list1[j] + "/" + ortho_name + "_predictions_small_AM.shp", driver='ESRI Shapefile') # turn this off if it's not needed

There seemed to be some problems with these shapefiles when they were examined in QGiS. The bounding boxes did not encompass the trees as often as one would expect based on the test results in 7_model_evaluation.

# Still, I try to use this map to validate how good the models are in terms of forestry domain-specific variables.